# QFD CMB Module Tutorial

This notebook provides an interactive tutorial for the QFD CMB Module, demonstrating how to compute CMB angular power spectra using photon-photon scattering models.

## Table of Contents
1. [Introduction](#introduction)
2. [Basic Power Spectrum Calculation](#basic-power-spectrum)
3. [CMB Angular Power Spectra](#cmb-spectra)
4. [Parameter Exploration](#parameter-exploration)
5. [Advanced Analysis](#advanced-analysis)

## 1. Introduction {#introduction}

The QFD CMB Module implements quantum field theory calculations for photon-photon scattering effects in the cosmic microwave background. This tutorial will guide you through the main features of the module.

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from qfd_cmb import oscillatory_psik, gaussian_window_chi, project_limber
from qfd_cmb.kernels import te_correlation_phase
from qfd_cmb.figures import plot_TT, plot_EE, plot_TE

# Set up plotting
plt.style.use('default')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

## 2. Basic Power Spectrum Calculation {#basic-power-spectrum}

Let's start by computing the oscillatory power spectrum that forms the basis of the QFD model.

In [ ]:
# Define wavenumber range
k = np.logspace(-4, 1, 200)  # 1/Mpc

# Compute power spectrum with default parameters
Pk_default = oscillatory_psik(k)

# Plot the result
plt.figure(figsize=(10, 6))
plt.loglog(k, Pk_default, 'b-', linewidth=2, label='QFD Power Spectrum')
plt.xlabel('k [1/Mpc]')
plt.ylabel('P(k)')
plt.title('Oscillatory Power Spectrum')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Power spectrum range: {Pk_default.min():.2e} to {Pk_default.max():.2e}")

### Interactive Parameter Exploration

Try changing the parameters below to see how they affect the power spectrum:

In [ ]:
# Interactive parameter exploration
# You can modify these values and re-run the cell

rpsi = 147.0      # Oscillation scale (Mpc)
Aosc = 0.55       # Oscillation amplitude
sigma_osc = 0.025 # Oscillation damping
ns = 0.96         # Spectral index

# Compute power spectrum with custom parameters
Pk_custom = oscillatory_psik(k, ns=ns, rpsi=rpsi, Aosc=Aosc, sigma_osc=sigma_osc)

# Compare with default
plt.figure(figsize=(12, 6))
plt.loglog(k, Pk_default, 'b-', linewidth=2, label='Default Parameters')
plt.loglog(k, Pk_custom, 'r--', linewidth=2, label=f'Custom (rpsi={rpsi}, Aosc={Aosc})')
plt.xlabel('k [1/Mpc]')
plt.ylabel('P(k)')
plt.title('Power Spectrum Comparison')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 3. CMB Angular Power Spectra {#cmb-spectra}

Now let's compute the CMB angular power spectra using the Limber projection.

In [ ]:
# Physical parameters (Planck-anchored)
lA = 301.0                    # Acoustic scale parameter
rpsi = 147.0                  # Oscillation scale (Mpc)
chi_star = lA * rpsi / np.pi  # Comoving distance to last scattering
sigma_chi = 250.0             # Width of last scattering surface

print(f"Physical parameters:")
print(f"  Acoustic scale lA: {lA}")
print(f"  Oscillation scale rpsi: {rpsi} Mpc")
print(f"  Distance to last scattering: {chi_star:.1f} Mpc")
print(f"  Last scattering width: {sigma_chi} Mpc")

# Set up visibility window
chi_grid = np.linspace(chi_star - 5*sigma_chi, chi_star + 5*sigma_chi, 300)
W_chi = gaussian_window_chi(chi_grid, chi_star, sigma_chi)

# Plot visibility window
plt.figure(figsize=(10, 6))
plt.plot(chi_grid, W_chi, 'g-', linewidth=2)
plt.axvline(chi_star, color='r', linestyle='--', alpha=0.7, label='Last scattering')
plt.xlabel('Comoving distance χ [Mpc]')
plt.ylabel('Visibility W(χ)')
plt.title('Last Scattering Visibility Window')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Compute CMB power spectra
ells = np.arange(2, 1500)

# Define power spectrum function
def Pk_func(k):
    return oscillatory_psik(k, ns=0.96, rpsi=rpsi, Aosc=0.55, sigma_osc=0.025)

print("Computing TT spectrum...")
Ctt = project_limber(ells, Pk_func, W_chi, chi_grid)

print("Computing EE spectrum...")
def Pk_EE(k):
    return 0.25 * Pk_func(k)  # EE is typically ~25% of TT amplitude

Cee = project_limber(ells, Pk_EE, W_chi, chi_grid)

print("Computing TE spectrum...")
rho = np.array([
    te_correlation_phase((ell + 0.5)/chi_star, rpsi, ell, chi_star) 
    for ell in ells
])
Cte = rho * np.sqrt(Ctt * Cee)

print("Spectra computed successfully!")

In [ ]:
# Plot all three spectra
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# TT spectrum
axes[0].loglog(ells, ells*(ells+1)*Ctt, 'r-', linewidth=2)
axes[0].set_xlabel(r'$\ell$')
axes[0].set_ylabel(r'$\ell(\ell+1)C_\ell^{TT}$')
axes[0].set_title('TT Spectrum')
axes[0].grid(True, alpha=0.3)

# EE spectrum
axes[1].loglog(ells, ells*(ells+1)*Cee, 'b-', linewidth=2)
axes[1].set_xlabel(r'$\ell$')
axes[1].set_ylabel(r'$\ell(\ell+1)C_\ell^{EE}$')
axes[1].set_title('EE Spectrum')
axes[1].grid(True, alpha=0.3)

# TE spectrum
sign = np.sign(Cte + 1e-30)
axes[2].semilogx(ells, sign * ells*(ells+1)*np.abs(Cte), 'g-', linewidth=2)
axes[2].axhline(0, color='k', linestyle='--', alpha=0.5)
axes[2].set_xlabel(r'$\ell$')
axes[2].set_ylabel(r'sign×$\ell(\ell+1)|C_\ell^{TE}|$')
axes[2].set_title('TE Spectrum')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print some statistics
print(f"TT spectrum peak: {(ells*(ells+1)*Ctt).max():.2e} at ell={ells[np.argmax(ells*(ells+1)*Ctt)]}")
print(f"EE spectrum peak: {(ells*(ells+1)*Cee).max():.2e} at ell={ells[np.argmax(ells*(ells+1)*Cee)]}")
print(f"TE correlation range: {rho.min():.3f} to {rho.max():.3f}")

## 4. Parameter Exploration {#parameter-exploration}

Let's explore how different parameters affect the CMB spectra.

In [ ]:
# Parameter sensitivity study
ells_short = np.arange(2, 800)  # Shorter range for faster computation

# Study rpsi dependence
rpsi_values = [140, 147, 154]
colors = ['blue', 'red', 'green']

plt.figure(figsize=(12, 8))

for i, rpsi_val in enumerate(rpsi_values):
    Pk_func = lambda k: oscillatory_psik(k, rpsi=rpsi_val, Aosc=0.55)
    Ctt_temp = project_limber(ells_short, Pk_func, W_chi, chi_grid)
    plt.loglog(ells_short, ells_short*(ells_short+1)*Ctt_temp, 
               color=colors[i], linewidth=2, label=f'rpsi = {rpsi_val} Mpc')

plt.xlabel(r'$\ell$')
plt.ylabel(r'$\ell(\ell+1)C_\ell^{TT}$')
plt.title('Oscillation Scale Sensitivity')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Oscillation amplitude sensitivity
Aosc_values = [0.0, 0.3, 0.55, 0.8]

plt.figure(figsize=(12, 8))

for Aosc_val in Aosc_values:
    Pk_func = lambda k: oscillatory_psik(k, rpsi=147.0, Aosc=Aosc_val)
    Ctt_temp = project_limber(ells_short, Pk_func, W_chi, chi_grid)
    plt.loglog(ells_short, ells_short*(ells_short+1)*Ctt_temp, 
               linewidth=2, label=f'Aosc = {Aosc_val}')

plt.xlabel(r'$\ell$')
plt.ylabel(r'$\ell(\ell+1)C_\ell^{TT}$')
plt.title('Oscillation Amplitude Sensitivity')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 5. Advanced Analysis {#advanced-analysis}

Finally, let's compare the QFD model with a standard model without oscillations.

In [ ]:
# Compare QFD model with standard model
Pk_standard = lambda k: oscillatory_psik(k, Aosc=0.0)  # No oscillations
Pk_qfd = lambda k: oscillatory_psik(k, rpsi=147.0, Aosc=0.55)

Ctt_standard = project_limber(ells_short, Pk_standard, W_chi, chi_grid)
Ctt_qfd = project_limber(ells_short, Pk_qfd, W_chi, chi_grid)

# Plot comparison
plt.figure(figsize=(14, 10))

# Main comparison
plt.subplot(2, 1, 1)
plt.loglog(ells_short, ells_short*(ells_short+1)*Ctt_standard, 'k-', 
           linewidth=2, label='Standard Model (no oscillations)')
plt.loglog(ells_short, ells_short*(ells_short+1)*Ctt_qfd, 'r-', 
           linewidth=2, label='QFD Model')
plt.xlabel(r'$\ell$')
plt.ylabel(r'$\ell(\ell+1)C_\ell^{TT}$')
plt.title('QFD vs Standard Model Comparison')
plt.legend()
plt.grid(True, alpha=0.3)

# Residual plot
plt.subplot(2, 1, 2)
residual = (Ctt_qfd - Ctt_standard) / Ctt_standard * 100
plt.semilogx(ells_short, residual, 'g-', linewidth=2)
plt.axhline(0, color='k', linestyle='--', alpha=0.5)
plt.xlabel(r'$\ell$')
plt.ylabel('Relative Difference [%]')
plt.title('QFD Model Residuals')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Maximum relative difference: {np.abs(residual).max():.2f}%")
print(f"RMS relative difference: {np.sqrt(np.mean(residual**2)):.2f}%")

## Summary

This tutorial has demonstrated:

1. **Basic power spectrum calculation** using the `oscillatory_psik` function
2. **CMB angular power spectra computation** using Limber projection
3. **Parameter sensitivity analysis** showing how different parameters affect the results
4. **Model comparison** between QFD and standard models

The QFD CMB Module provides a comprehensive framework for studying photon-photon scattering effects in the cosmic microwave background. The oscillatory features in the power spectrum lead to distinctive signatures in the CMB angular power spectra that can be observationally tested.

For more advanced usage, including parameter fitting and MCMC analysis, see the `advanced_fitting.py` example script.